### Phenotype Table for Initial Set of Training Data

In [1]:
import datetime
import pandas as pd
import numpy as np

#### Import dataset downloaded from betydb in R

In [2]:
df_0 = pd.read_csv('../data/raw/mac_season_4.csv')
df_0.head()

/opt/miniconda3/envs/terraref-datasets/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (18,26,32,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,checked,result_type,id,citation_id,site_id,treatment_id,sitename,city,lat,...,n,statname,stat,notes,access_level,cultivar,entity,method_name,view_url,edit_url
0,1,0,traits,6001958927,6.000000e+09,6000005673,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 5,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI181083,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
1,2,0,traits,6001958928,6.000000e+09,6000005676,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 6,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI564163,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
2,3,0,traits,6001958931,6.000000e+09,6000005685,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 9,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI52606,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
3,4,0,traits,6001958933,6.000000e+09,6000005691,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 11,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI533792,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
4,5,0,traits,6001958936,6.000000e+09,6000005700,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 14,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI535794,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...


In [3]:
df_0.columns

Index(['Unnamed: 0', 'checked', 'result_type', 'id', 'citation_id', 'site_id',
       'treatment_id', 'sitename', 'city', 'lat', 'lon', 'scientificname',
       'commonname', 'genus', 'species_id', 'cultivar_id', 'author',
       'citation_year', 'treatment', 'date', 'time', 'raw_date', 'month',
       'year', 'dateloc', 'trait', 'trait_description', 'mean', 'units', 'n',
       'statname', 'stat', 'notes', 'access_level', 'cultivar', 'entity',
       'method_name', 'view_url', 'edit_url'],
      dtype='object')

In [4]:
df_1 = df_0.drop(labels=['Unnamed: 0', 'checked', 'citation_id', 'city', 'scientificname', 'commonname', 'genus',
                        'species_id', 'author', 'citation_year', 'trait_description', 'units', 'n', 'statname',
                        'stat', 'access_level', 'view_url', 'edit_url'], axis=1)

df_1.head()

,result_type,id,site_id,treatment_id,sitename,lat,lon,cultivar_id,treatment,date,...,raw_date,month,year,dateloc,trait,mean,notes,cultivar,entity,method_name
0,traits,6001958927,6000005673,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 5,33.074907,-111.974982,6000000730,"BAP 2017, water-deficit stress Aug 1-14",2017 Jun 14 (America/Phoenix),...,2017-06-15 00:00:00 -0500,6,2017,5.0,leaf_desiccation_present,0.0,NaN,PI181083,NaN,Visual assessment of leaf dessication
1,traits,6001958928,6000005676,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 6,33.074907,-111.974966,6000000231,"BAP 2017, water-deficit stress Aug 1-14",2017 Jun 14 (America/Phoenix),...,2017-06-15 00:00:00 -0500,6,2017,5.0,leaf_desiccation_present,0.0,NaN,PI564163,NaN,Visual assessment of leaf dessication
2,traits,6001958931,6000005685,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 9,33.074907,-111.974917,6000000860,"BAP 2017, water-deficit stress Aug 1-14",2017 Jun 14 (America/Phoenix),...,2017-06-15 00:00:00 -0500,6,2017,5.0,leaf_desiccation_present,0.0,NaN,PI52606,NaN,Visual assessment of leaf dessication
3,traits,6001958933,6000005691,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 11,33.074907,-111.974884,6000000863,"BAP 2017, water-deficit stress Aug 1-14",2017 Jun 14 (America/Phoenix),...,2017-06-15 00:00:00 -0500,6,2017,5.0,leaf_desiccation_present,0.0,NaN,PI533792,NaN,Visual assessment of leaf dessication
4,traits,6001958936,6000005700,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 14,33.074907,-111.974835,6000000869,"BAP 2017, water-deficit stress Aug 1-14",2017 Jun 14 (America/Phoenix),...,2017-06-15 00:00:00 -0500,6,2017,5.0,leaf_desiccation_present,0.0,NaN,PI535794,NaN,Visual assessment of leaf dessication


In [5]:
timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
output_filename = f'pheno-table_{timestamp}.csv'.replace(':', '')
df_1.to_csv(f'../data/processed/{output_filename}', index=False)